In [ ]:
pip install openmeteo-requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.7/684.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.12.19
    Uninstalling flatbuffers-25.12.19:
      Successfully uninstalled flatbuffers-25.12.19


In [ ]:
pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 2.5 MB/s eta 0:00:00


Tokyo Saving


In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 35.6895,
	"longitude": 139.6917,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide", "nitrogen_dioxide", "ammonia", "methane", "sulphur_dioxide", "ozone"],
	"start_date": "2020-01-01",
	"end_date": "2025-01-01",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
hourly_carbon_dioxide = hourly.Variables(3).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_ammonia = hourly.Variables(5).ValuesAsNumpy()
hourly_methane = hourly.Variables(6).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(7).ValuesAsNumpy()
hourly_ozone = hourly.Variables(8).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["carbon_dioxide"] = hourly_carbon_dioxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["ammonia"] = hourly_ammonia
hourly_data["methane"] = hourly_methane
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["ozone"] = hourly_ozone

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)


Coordinates: 35.70000457763672°N 139.70001220703125°E
Elevation: 40.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                            date       pm10      pm2_5  carbon_monoxide  \
0     2020-01-01 00:00:00+00:00        NaN        NaN              NaN   
1     2020-01-01 01:00:00+00:00        NaN        NaN              NaN   
2     2020-01-01 02:00:00+00:00        NaN        NaN              NaN   
3     2020-01-01 03:00:00+00:00        NaN        NaN              NaN   
4     2020-01-01 04:00:00+00:00        NaN        NaN              NaN   
...                         ...        ...        ...              ...   
43867 2025-01-01 19:00:00+00:00  41.599998  41.099998            512.0   
43868 2025-01-01 20:00:00+00:00  42.500000  42.000000            494.0   
43869 2025-01-01 21:00:00+00:00  51.799999  51.299999            493.0   
43870 2025-01-01 22:00:00+00:00  70.300003  69.800003            508.0   
43871 2025-01-01 23:00:00+00:00  84.099998  83.599998          

In [ ]:
hourly_dataframe.to_csv('Tokyo.csv', index=False)
print('Dataset saved to Tokyo.csv')

Dataset saved to Tokyo.csv


Delhi Saving

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 28.6519,
	"longitude": 77.2315,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide", "nitrogen_dioxide", "ammonia", "methane", "sulphur_dioxide", "ozone"],
	"start_date": "2020-01-01",
	"end_date": "2025-01-01",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
hourly_carbon_dioxide = hourly.Variables(3).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_ammonia = hourly.Variables(5).ValuesAsNumpy()
hourly_methane = hourly.Variables(6).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(7).ValuesAsNumpy()
hourly_ozone = hourly.Variables(8).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["carbon_dioxide"] = hourly_carbon_dioxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["ammonia"] = hourly_ammonia
hourly_data["methane"] = hourly_methane
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["ozone"] = hourly_ozone

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)


Coordinates: 28.70000457763672°N 77.20001220703125°E
Elevation: 231.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                            date        pm10       pm2_5  carbon_monoxide  \
0     2020-01-01 00:00:00+00:00         NaN         NaN              NaN   
1     2020-01-01 01:00:00+00:00         NaN         NaN              NaN   
2     2020-01-01 02:00:00+00:00         NaN         NaN              NaN   
3     2020-01-01 03:00:00+00:00         NaN         NaN              NaN   
4     2020-01-01 04:00:00+00:00         NaN         NaN              NaN   
...                         ...         ...         ...              ...   
43867 2025-01-01 19:00:00+00:00  170.399994  167.600006           1393.0   
43868 2025-01-01 20:00:00+00:00  165.800003  163.000000           1328.0   
43869 2025-01-01 21:00:00+00:00  160.699997  157.899994           1284.0   
43870 2025-01-01 22:00:00+00:00  154.600006  151.800003           1274.0   
43871 2025-01-01 23:00:00+00:00  149.5000

In [ ]:
hourly_dataframe.to_csv('Delhi.csv', index=False)
print('Dataset saved to Delhi.csv')

Dataset saved to Delhi.csv


Shanghai Saving

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 31.2222,
	"longitude": 121.4581,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide", "nitrogen_dioxide", "ammonia", "methane", "sulphur_dioxide", "ozone"],
	"start_date": "2020-01-01",
	"end_date": "2025-01-01",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
hourly_carbon_dioxide = hourly.Variables(3).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_ammonia = hourly.Variables(5).ValuesAsNumpy()
hourly_methane = hourly.Variables(6).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(7).ValuesAsNumpy()
hourly_ozone = hourly.Variables(8).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["carbon_dioxide"] = hourly_carbon_dioxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["ammonia"] = hourly_ammonia
hourly_data["methane"] = hourly_methane
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["ozone"] = hourly_ozone

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)


Coordinates: 31.20000457763672°N 121.5°E
Elevation: 7.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                            date       pm10      pm2_5  carbon_monoxide  \
0     2020-01-01 00:00:00+00:00        NaN        NaN              NaN   
1     2020-01-01 01:00:00+00:00        NaN        NaN              NaN   
2     2020-01-01 02:00:00+00:00        NaN        NaN              NaN   
3     2020-01-01 03:00:00+00:00        NaN        NaN              NaN   
4     2020-01-01 04:00:00+00:00        NaN        NaN              NaN   
...                         ...        ...        ...              ...   
43867 2025-01-01 19:00:00+00:00  72.900002  68.500000            774.0   
43868 2025-01-01 20:00:00+00:00  66.099998  61.700001            888.0   
43869 2025-01-01 21:00:00+00:00  60.299999  56.000000           1158.0   
43870 2025-01-01 22:00:00+00:00  60.500000  56.200001           1720.0   
43871 2025-01-01 23:00:00+00:00  85.699997  81.699997           2440.0   

  

In [ ]:
hourly_dataframe.to_csv('Shanghai.csv', index=False)
print('Dataset saved to Shanghai.csv')

Dataset saved to Shanghai.csv


Dhaka Saving

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 23.7104,
	"longitude": 90.4074,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide", "nitrogen_dioxide", "ammonia", "methane", "sulphur_dioxide", "ozone"],
	"start_date": "2020-01-01",
	"end_date": "2025-01-01",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
hourly_carbon_dioxide = hourly.Variables(3).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_ammonia = hourly.Variables(5).ValuesAsNumpy()
hourly_methane = hourly.Variables(6).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(7).ValuesAsNumpy()
hourly_ozone = hourly.Variables(8).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["carbon_dioxide"] = hourly_carbon_dioxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["ammonia"] = hourly_ammonia
hourly_data["methane"] = hourly_methane
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["ozone"] = hourly_ozone

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)


Coordinates: 23.70000457763672°N 90.39999389648438°E
Elevation: 27.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                            date       pm10      pm2_5  carbon_monoxide  \
0     2020-01-01 00:00:00+00:00        NaN        NaN              NaN   
1     2020-01-01 01:00:00+00:00        NaN        NaN              NaN   
2     2020-01-01 02:00:00+00:00        NaN        NaN              NaN   
3     2020-01-01 03:00:00+00:00        NaN        NaN              NaN   
4     2020-01-01 04:00:00+00:00        NaN        NaN              NaN   
...                         ...        ...        ...              ...   
43867 2025-01-01 19:00:00+00:00  94.500000  93.400002            904.0   
43868 2025-01-01 20:00:00+00:00  92.199997  91.300003            830.0   
43869 2025-01-01 21:00:00+00:00  90.300003  89.400002            776.0   
43870 2025-01-01 22:00:00+00:00  88.900002  88.000000            762.0   
43871 2025-01-01 23:00:00+00:00  88.000000  87.300003           

In [ ]:
hourly_dataframe.to_csv('Dhaka.csv', index=False)
print('Dataset saved to Dhaka.csv')

Dataset saved to Dhaka.csv


Beijing Saving

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 39.9075,
	"longitude": 116.3972,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide", "nitrogen_dioxide", "ammonia", "methane", "sulphur_dioxide", "ozone"],
	"start_date": "2020-01-01",
	"end_date": "2025-01-01",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
hourly_carbon_dioxide = hourly.Variables(3).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_ammonia = hourly.Variables(5).ValuesAsNumpy()
hourly_methane = hourly.Variables(6).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(7).ValuesAsNumpy()
hourly_ozone = hourly.Variables(8).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["carbon_dioxide"] = hourly_carbon_dioxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["ammonia"] = hourly_ammonia
hourly_data["methane"] = hourly_methane
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["ozone"] = hourly_ozone

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)


Coordinates: 39.90000915527344°N 116.39999389648438°E
Elevation: 47.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                            date       pm10      pm2_5  carbon_monoxide  \
0     2020-01-01 00:00:00+00:00        NaN        NaN              NaN   
1     2020-01-01 01:00:00+00:00        NaN        NaN              NaN   
2     2020-01-01 02:00:00+00:00        NaN        NaN              NaN   
3     2020-01-01 03:00:00+00:00        NaN        NaN              NaN   
4     2020-01-01 04:00:00+00:00        NaN        NaN              NaN   
...                         ...        ...        ...              ...   
43867 2025-01-01 19:00:00+00:00  62.900002  60.500000            365.0   
43868 2025-01-01 20:00:00+00:00  50.099998  48.400002            369.0   
43869 2025-01-01 21:00:00+00:00  46.700001  45.500000            492.0   
43870 2025-01-01 22:00:00+00:00  50.900002  49.900002            821.0   
43871 2025-01-01 23:00:00+00:00  77.300003  76.599998          

In [ ]:
hourly_dataframe.to_csv('Beijing.csv', index=False)
print('Dataset saved to Beijing.csv')

Dataset saved to Beijing.csv
